<a href="https://colab.research.google.com/github/MichelyTamessawa/classificacao-laranja/blob/master/classificacao_laranjas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler

# **Leitura do Dataset**

In [ ]:
def leitura(diretorio):
  siglas_laranjas = ["B", "C", "D", "P", "V"]
  classes_laranjas = ["Boa", "Casca Grossa", "Dano Praga", "Podre", "Verde"]
  caminho_drive = "/content/drive/MyDrive/4 ano/tcc/classificação de laranjas/"

  # inicialização dos objetos do tipo DataFrame que vão armazenar os dados da base de dados
  boa = pd.DataFrame()
  casca_grossa = pd.DataFrame()
  dano_praga = pd.DataFrame()
  podre = pd.DataFrame()
  verde = pd.DataFrame()

  for i, classe in enumerate(classes_laranjas):
    for j in range(1, 11):

      # caminho de diretórios até o arquivo da base  
      caminho = caminho_drive + diretorio + classe + "/Fold " + str(j) + siglas_laranjas[i] + ".csv"
      
      # leiturclassea de uma linha do arquivo 
      linha = pd.read_csv(caminho)
      
      # a linha que foi lida é adicionada ao DataFrame da sua classe
      if classe == "Boa":
          boa = pd.concat([boa, linha],ignore_index=True)
      elif classe == "Casca Grossa":
          casca_grossa = pd.concat([casca_grossa, linha],ignore_index=True)
      elif classe == "Dano Praga":
          dano_praga = pd.concat([dano_praga, linha],ignore_index=True)
      elif classe == "Podre":
          podre = pd.concat([podre, linha],ignore_index=True)
      elif classe == "Verde":
          verde = pd.concat([verde, linha],ignore_index=True)
  
  return boa, casca_grossa, dano_praga, podre, verde

# **Classificação Multiclasses**

In [ ]:
def todos_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  df = pd.concat([boa, casca_grossa, dano_praga, podre, verde], axis=0, ignore_index=True)
  
  X = df.drop("class", axis=1)
  y = df["class"]
  
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
  
  return x_train, x_test, y_train, y_test

# **Classificação Binária: (Um-vs-Todos)**

## Divisão de inteiros por 4

In [ ]:
def divide_quatro_partes(num):
  resto = num % 4

  if resto == 0:
      return [num//4, num//4, num//4, num//4]
  elif resto == 3:
      return [num//4+1, num//4+1, num//4+1, num//4]
  elif resto == 2:
      return [num//4+1, num//4+1, num//4, num//4]
  elif resto == 1:
      return [num//4+1, num//4, num//4, num//4]

## Divisão de inteiros por 3

In [ ]:
def divide_tres_partes(num):
    resto = num % 3

    if resto == 0:
        return [num//3, num//3, num//3]
    elif resto == 2:
        return [num//3+1, num//3+1, num//3]
    elif resto == 1:
        return [num//3+1, num//3, num//3]

## Balanceamento das classes

In [ ]:
def boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  n_classe  = boa.shape[0]
  divisao_classes = divide_quatro_partes(n_classe)

  c = casca_grossa.sample(n=divisao_classes[0],random_state=1)
  d = dano_praga.sample(n=divisao_classes[1],random_state=1)
  p = podre.sample(n=divisao_classes[2],random_state=1)
  v = verde.sample(n=divisao_classes[3],random_state=1)

  df = pd.concat([boa, c, d, p, v], axis=0, ignore_index=True)
  df["class"].replace({2:1, 3:1, 4:1}, inplace=True)

  X = df.drop("class", axis=1)
  y = df["class"]

  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
  
  return x_train, x_test, y_train, y_test


def casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  n_classe  = casca_grossa.shape[0]
  divisao_classes = divide_quatro_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  d = dano_praga.sample(n=divisao_classes[1],random_state=1)
  p = podre.sample(n=divisao_classes[2],random_state=1)
  v = verde.sample(n=divisao_classes[3],random_state=1)
  
  df = pd.concat([casca_grossa, b, d, p, v], axis=0, ignore_index=True)
  df["class"].replace({0:1, 1:0, 2:1, 3:1, 4:1}, inplace=True)
  
  X = df.drop("class", axis=1)
  y = df["class"]

  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
  
  return x_train, x_test, y_train, y_test


def dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  n_classe  = dano_praga.shape[0]
  divisao_classes = divide_quatro_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  c = casca_grossa.sample(n=divisao_classes[1],random_state=1)
  p = podre.sample(n=divisao_classes[2],random_state=1)
  v = verde.sample(n=divisao_classes[3],random_state=1)
    
  df = pd.concat([dano_praga, c, b, p, v], axis=0, ignore_index=True)
  df["class"].replace({0:1, 2:0, 3:1, 4:1}, inplace=True)
  
  X = df.drop("class", axis=1)
  y = df["class"]
      
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

  return x_train, x_test, y_train, y_test


def podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  n_classe  = podre.shape[0]
  divisao_classes = divide_quatro_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  c = casca_grossa.sample(n=divisao_classes[1],random_state=1)
  d = dano_praga.sample(n=divisao_classes[2],random_state=1)
  v = verde.sample(n=divisao_classes[3],random_state=1)
    
  df = pd.concat([podre, c, b, d, v], axis=0, ignore_index=True)
  df["class"].replace({0:1, 2:1, 3:0, 4:1}, inplace=True)

  X = df.drop("class", axis=1)
  y = df["class"]
      
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

  return x_train, x_test, y_train, y_test


def verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  n_classe  = verde.shape[0]
  divisao_classes = divide_quatro_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  c = casca_grossa.sample(n=divisao_classes[1],random_state=1)
  d = dano_praga.sample(n=divisao_classes[2],random_state=1)
  p = podre.sample(n=divisao_classes[3],random_state=1)

  df = pd.concat([verde, c, b, p, d], axis=0, ignore_index=True)
  df["class"].replace({0:1, 2:1, 3:1, 4:0}, inplace=True)  

  X = df.drop("class", axis=1)
  y = df["class"]

  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

  return x_train, x_test, y_train, y_test

## Balanceamento das classes removendo a Verde (RV)

In [ ]:
def boa_vs_todosRV(boa, casca_grossa, dano_praga, podre):
  n_classe  = boa.shape[0]
  divisao_classes = divide_tres_partes(n_classe)

  c = casca_grossa.sample(n=divisao_classes[0],random_state=1)
  d = dano_praga.sample(n=divisao_classes[1],random_state=1)
  p = podre.sample(n=divisao_classes[2],random_state=1)

  df = pd.concat([boa, c, d, p], axis=0, ignore_index=True)
  df["class"].replace({2:1, 3:1}, inplace=True)
  
  X = df.drop("class", axis=1)
  y = df["class"]

  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
  
  return x_train, x_test, y_train, y_test


def casca_grossa_vs_todosRV(boa, casca_grossa, dano_praga, podre):
  n_classe  = casca_grossa.shape[0]
  divisao_classes = divide_tres_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  d = dano_praga.sample(n=divisao_classes[1],random_state=1)
  p = podre.sample(n=divisao_classes[2],random_state=1)

  df = pd.concat([casca_grossa, b, d, p], axis=0, ignore_index=True)
  df["class"].replace({0:1, 1:0, 2:1, 3:1}, inplace=True)
  
  X = df.drop("class", axis=1)
  y = df["class"]

  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
  
  return x_train, x_test, y_train, y_test


def dano_praga_vs_todosRV(boa, casca_grossa, dano_praga, podre):
  n_classe  = dano_praga.shape[0]
  divisao_classes = divide_tres_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  c = casca_grossa.sample(n=divisao_classes[1],random_state=1)
  p = podre.sample(n=divisao_classes[2],random_state=1)
    
  df = pd.concat([dano_praga, c, b, p], axis=0, ignore_index=True)
  df["class"].replace({0:1, 2:0, 3:1}, inplace=True)
  
  X = df.drop("class", axis=1)
  y = df["class"]
      
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

  return x_train, x_test, y_train, y_test


def podre_vs_todosRV(boa, casca_grossa, dano_praga, podre):
  n_classe  = podre.shape[0]
  divisao_classes = divide_tres_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  c = casca_grossa.sample(n=divisao_classes[1],random_state=1)
  d = dano_praga.sample(n=divisao_classes[2],random_state=1)
      
  df = pd.concat([podre, c, b, d], axis=0, ignore_index=True)
  df["class"].replace({0:1, 2:1, 3:0}, inplace=True)

  X = df.drop("class", axis=1)
  y = df["class"]
      
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

  return x_train, x_test, y_train, y_test

# **Classificadores**

## K-NN

In [ ]:
def knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names):
  # Normalizacao dos dados
  scaler = MinMaxScaler()
  X_train_scaled = scaler.fit_transform(x_train)
  X_test_scaled = scaler.transform(x_test)
  
  knn = KNeighborsClassifier()
  
  param_grid = {
      'n_neighbors': range(1,30),
      'leaf_size': (20,40,1),
      'p': (1,2),
      'weights': ('uniform', 'distance'),
      'metric': ('minkowski', 'chebyshev') }

  # Treinamento com gridsearch
  grid = GridSearchCV(estimator=knn, param_grid=param_grid, cv=10, refit='True', scoring='accuracy')
  grid_search = grid.fit(X_train_scaled, y_train)
  print("Parametros: {}".format(grid_search.best_params_))
  
  # Teste
  y_test_result = grid.predict(X_test_scaled)
  test_accuracy = accuracy_score(y_test, y_test_result)*100
  print("Acuracia dos testes: {:.2f}%".format(test_accuracy))

  # Matriz de Confusao
  print("Matriz de Confusao:")
  print (pd.crosstab(y_test, y_test_result, rownames=['Real'], colnames=['Predito'], margins=True))

  # Relatorio de Classificacao 
  print("Relatorio de Classificacao:")
  print(classification_report(y_test, y_test_result, target_names=target_names))

## Random Forest

In [ ]:
def rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names):
  # Normalizacao dos dados
  scaler = MinMaxScaler()
  X_train_scaled = scaler.fit_transform(x_train)
  X_test_scaled = scaler.transform(x_test)

  rf = RandomForestClassifier(random_state=0)
  
  param_grid = {
    'max_depth': [10, 50],
    'min_samples_leaf': [1,3],
    'min_samples_split': [2,5],
    'n_estimators': [100, 200]
  }

  # Treinamento com gridsearch
  grid = GridSearchCV(rf, param_grid, cv=10, refit='True', scoring='accuracy')
  grid_search = grid.fit(X_train_scaled, y_train)
  print("Parametros: {}".format(grid_search.best_params_))

  # Teste
  y_test_result = grid.predict(X_test_scaled)
  test_accuracy = accuracy_score(y_test, y_test_result)*100
  print("Acuracia dos testes: {:.2f}%".format(test_accuracy))

  # Matriz de Confusao
  print("Matriz de Confusao:")
  print (pd.crosstab(y_test, y_test_result, rownames=['Real'], colnames=['Predito'], margins=True))

  # Relatorio de Classificacao 
  print("Relatorio de Classificacao:")
  print(classification_report(y_test, y_test_result, target_names=target_names))


## Classificador SVM

In [ ]:
def svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names):
  # Normalizacao dos dados
  scaler = MinMaxScaler()
  X_train_scaled = scaler.fit_transform(x_train)
  X_test_scaled = scaler.transform(x_test)

  svm = SVC()
  
  param_grid = [
    {"kernel": ['rbf'], "gamma": [1, 0.1, 0.01, 0.001, 0.0001], "C": [0.1, 1, 10, 100, 1000]},
    {"kernel": ["linear"], "C": [1, 10, 100, 1000]},
  ]
              
  # Treinamento com gridsearch
  grid = GridSearchCV(svm, param_grid, cv=10, refit=True, scoring='accuracy', n_jobs=-1)
  grid_search = grid.fit(X_train_scaled, y_train)
  print("Parametros: {}".format(grid_search.best_params_))

  # Teste
  y_test_result = grid.predict(X_test_scaled)
  test_accuracy = accuracy_score(y_test, y_test_result)*100
  print("Acuracia dos testes: {:.2f}%".format(test_accuracy))

  # Matriz de Confusao
  print("Matriz de Confusao:")
  print (pd.crosstab(y_test, y_test_result, rownames=['Real'], colnames=['Predito'], margins=True))

  # Relatorio de Classificacao 
  print("Relatorio de Classificacao:")
  print(classification_report(y_test, y_test_result, target_names=target_names))

# **>> Execuções <<**

# **Multiclasses**

## K-NN

### Tons de Cinza

In [ ]:
  target_names = ['0','1','2','3','4']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parametros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 6, 'p': 1, 'weights': 'distance'}
Acuracia dos testes: 71.85%
Matriz de Confusao:
Predito    0    1    2    3    4   All
Real                                  
0        333   51   32   35   19   470
1         27  343   39   40    5   454
2         36   36  331   27   30   460
3         40   52   41  277   24   434
4         32    9   43   21  347   452
All      468  491  486  400  425  2270
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.71      0.71      0.71       470
           1       0.70      0.76      0.73       454
           2       0.68      0.72      0.70       460
           3       0.69      0.64      0.66       434
           4       0.82      0.77      0.79       452

    accuracy                           0.72      2270
   macro avg       0.72      0.72      0.72      2270
weighted avg       0.72      0.72      0.72      2270



### RGB

In [ ]:
target_names = ['0','1','2','3','4']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parametros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}
Acuracia dos testes: 88.15%
Matriz de Confusao:
Predito    0    1    2    3    4   All
Real                                  
0        404   18   24   18    6   470
1         10  405   12   23    4   454
2         18    6  406   29    1   460
3         30   24   33  346    1   434
4          7    1    2    2  440   452
All      469  454  477  418  452  2270
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.86      0.86      0.86       470
           1       0.89      0.89      0.89       454
           2       0.85      0.88      0.87       460
           3       0.83      0.80      0.81       434
           4       0.97      0.97      0.97       452

    accuracy                           0.88      2270
   macro avg       0.88      0.88      0.88      2270
weighted avg       0.88      0.88      0.88      2270



## RandomForest

### Tons de Cinza

In [ ]:
target_names = ['0','1','2','3','4']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parametros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acuracia dos testes: 74.80%
Matriz de Confusao:
Predito    0    1    2    3    4   All
Real                                  
0        361   42   29   27   11   470
1         28  356   41   25    4   454
2         26   28  353   26   27   460
3         31   60   44  276   23   434
4         25    9   46   20  352   452
All      471  495  513  374  417  2270
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.77      0.77      0.77       470
           1       0.72      0.78      0.75       454
           2       0.69      0.77      0.73       460
           3       0.74      0.64      0.68       434
           4       0.84      0.78      0.81       452

    accuracy                           0.75      2270
   macro avg       0.75      0.75      0.75      2270
weighted avg       0.75      0.75      0.75      2270



### RGB

In [ ]:
target_names = ['0','1','2','3','4']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parametros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acuracia dos testes: 86.56%
Matriz de Confusao:
Predito    0    1    2    3    4   All
Real                                  
0        380   24   37   25    4   470
1         14  414   13   13    0   454
2         15   12  398   34    1   460
3         24   33   50  327    0   434
4          2    4    0    0  446   452
All      435  487  498  399  451  2270
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.87      0.81      0.84       470
           1       0.85      0.91      0.88       454
           2       0.80      0.87      0.83       460
           3       0.82      0.75      0.79       434
           4       0.99      0.99      0.99       452

    accuracy                           0.87      2270
   macro avg       0.87      0.87      0.86      2270
weighted avg       0.87      0.87      0.87      2270



## SVM

### Tons de Cinza

In [ ]:
target_names = ['0','1','2','3','4']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parametros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acuracia dos testes: 77.58%
Matriz de Confusao:
Predito    0    1    2    3    4   All
Real                                  
0        349   47   32   30   12   470
1         30  378   16   25    5   454
2         28   24  382   21    5   460
3         38   55   42  287   12   434
4         26   12   10   39  365   452
All      471  516  482  402  399  2270
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.74      0.74      0.74       470
           1       0.73      0.83      0.78       454
           2       0.79      0.83      0.81       460
           3       0.71      0.66      0.69       434
           4       0.91      0.81      0.86       452

    accuracy                           0.78      2270
   macro avg       0.78      0.77      0.78      2270
weighted avg       0.78      0.78      0.78      2270



### RGB

In [ ]:
target_names = ['0','1','2','3','4']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parametros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acuracia dos testes: 93.96%
Matriz de Confusao:
Predito    0    1    2    3    4   All
Real                                  
0        431    7   16   14    2   470
1          7  436    5    4    2   454
2         17    5  427   11    0   460
3         24    8   13  389    0   434
4          0    1    1    0  450   452
All      479  457  462  418  454  2270
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.90      0.92      0.91       470
           1       0.95      0.96      0.96       454
           2       0.92      0.93      0.93       460
           3       0.93      0.90      0.91       434
           4       0.99      1.00      0.99       452

    accuracy                           0.94      2270
   macro avg       0.94      0.94      0.94      2270
weighted avg       0.94      0.94      0.94      2270



# **Multiclasses: Removendo a Verde**

## K-NN

### Tons de Cinza

In [ ]:
target_names = ['0','1','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre,  verde[0:0])
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parametros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
Acuracia dos testes: 72.41%
Matriz de Confusao:
Predito    0    1    2    3   All
Real                             
0        355   45   36   28   464
1         35  329   42   45   451
2         34   48  350   31   463
3         54   53   50  281   438
All      478  475  478  385  1816
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.74      0.77      0.75       464
           1       0.69      0.73      0.71       451
           2       0.73      0.76      0.74       463
           3       0.73      0.64      0.68       438

    accuracy                           0.72      1816
   macro avg       0.72      0.72      0.72      1816
weighted avg       0.72      0.72      0.72      1816



### RGB

In [ ]:
target_names = ['0','1','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre,  verde[0:0])
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parametros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}
Acuracia dos testes: 86.40%
Matriz de Confusao:
Predito    0    1    2    3   All
Real                             
0        396   25   21   22   464
1         21  401   14   15   451
2         24    8  399   32   463
3         30   16   19  373   438
All      471  450  453  442  1816
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.84      0.85      0.85       464
           1       0.89      0.89      0.89       451
           2       0.88      0.86      0.87       463
           3       0.84      0.85      0.85       438

    accuracy                           0.86      1816
   macro avg       0.86      0.86      0.86      1816
weighted avg       0.86      0.86      0.86      1816



## RandomForest

### Tons de Cinza

In [ ]:
target_names = ['0','1','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre,  verde[0:0])
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parametros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Acuracia dos testes: 76.05%
Matriz de Confusao:
Predito    0    1    2    3   All
Real                             
0        355   47   36   26   464
1         27  342   43   39   451
2         20   36  384   23   463
3         52   46   40  300   438
All      454  471  503  388  1816
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.78      0.77      0.77       464
           1       0.73      0.76      0.74       451
           2       0.76      0.83      0.80       463
           3       0.77      0.68      0.73       438

    accuracy                           0.76      1816
   macro avg       0.76      0.76      0.76      1816
weighted avg       0.76      0.76      0.76      1816



### RGB

In [ ]:
target_names = ['0','1','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre,  verde[0:0])
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parametros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acuracia dos testes: 85.90%
Matriz de Confusao:
Predito    0    1    2    3   All
Real                             
0        380   27   33   24   464
1         16  402   18   15   451
2         15    8  418   22   463
3         27   22   29  360   438
All      438  459  498  421  1816
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.87      0.82      0.84       464
           1       0.88      0.89      0.88       451
           2       0.84      0.90      0.87       463
           3       0.86      0.82      0.84       438

    accuracy                           0.86      1816
   macro avg       0.86      0.86      0.86      1816
weighted avg       0.86      0.86      0.86      1816



## SVM

### Tons de Cinza

In [ ]:
target_names = ['0','1','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre,  verde[0:0])
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parametros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acuracia dos testes: 78.30%
Matriz de Confusao:
Predito    0    1    2    3   All
Real                             
0        355   46   31   32   464
1         34  363   24   30   451
2         22   22  393   26   463
3         43   49   35  311   438
All      454  480  483  399  1816
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.78      0.77      0.77       464
           1       0.76      0.80      0.78       451
           2       0.81      0.85      0.83       463
           3       0.78      0.71      0.74       438

    accuracy                           0.78      1816
   macro avg       0.78      0.78      0.78      1816
weighted avg       0.78      0.78      0.78      1816



### RGB

In [ ]:
target_names = ['0','1','2','3']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
x_train, x_test, y_train, y_test = todos_vs_todos(boa, casca_grossa, dano_praga, podre,  verde[0:0])
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Parametros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acuracia dos testes: 94.16%
Matriz de Confusao:
Predito    0    1    2    3   All
Real                             
0        426    8   21    9   464
1         12  428    5    6   451
2          8    3  443    9   463
3         11    5    9  413   438
All      457  444  478  437  1816
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.93      0.92      0.93       464
           1       0.96      0.95      0.96       451
           2       0.93      0.96      0.94       463
           3       0.95      0.94      0.94       438

    accuracy                           0.94      1816
   macro avg       0.94      0.94      0.94      1816
weighted avg       0.94      0.94      0.94      1816



# **Binária**

## K-NN

### Tons de Cinza

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Verde vs Todos")
x_train, x_test, y_train, y_test = verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parametros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
Acuracia dos testes: 78.96%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        397   74  471
1        117  320  437
All      514  394  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.77      0.84      0.81       471
           1       0.81      0.73      0.77       437

    accuracy                           0.79       908
   macro avg       0.79      0.79      0.79       908
weighted avg       0.79      0.79      0.79       908



Casca Grossa vs Todos
Parametros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 4, 'p': 1, 'weights': 'distance'}
Acuracia dos testes: 85.35%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        425   46  471
1         87  350  437
All      512  396  908
Relatorio de Classificacao:
              precision    recall  f1-score   supp

### RGB

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Verde vs Todos")
x_train, x_test, y_train, y_test = verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parametros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}
Acuracia dos testes: 86.89%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        423   48  471
1         71  366  437
All      494  414  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       471
           1       0.88      0.84      0.86       437

    accuracy                           0.87       908
   macro avg       0.87      0.87      0.87       908
weighted avg       0.87      0.87      0.87       908



Casca Grossa vs Todos
Parametros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 4, 'p': 1, 'weights': 'distance'}
Acuracia dos testes: 93.83%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        459   12  471
1         44  393  437
All      503  405  908
Relatorio de Classificacao:
              precision    recall  f1-score   suppo

## RandomForest

### Tons de Cinza

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Verde vs Todos")
x_train, x_test, y_train, y_test = verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parametros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acuracia dos testes: 82.05%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        391   80  471
1         83  354  437
All      474  434  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.82      0.83      0.83       471
           1       0.82      0.81      0.81       437

    accuracy                           0.82       908
   macro avg       0.82      0.82      0.82       908
weighted avg       0.82      0.82      0.82       908



Casca Grossa vs Todos
Parametros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acuracia dos testes: 85.13%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        418   53  471
1         82  355  437
All      500  408  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

   

### RGB

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Verde vs Todos")
x_train, x_test, y_train, y_test = verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parametros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Acuracia dos testes: 87.33%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        407   64  471
1         51  386  437
All      458  450  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.89      0.86      0.88       471
           1       0.86      0.88      0.87       437

    accuracy                           0.87       908
   macro avg       0.87      0.87      0.87       908
weighted avg       0.87      0.87      0.87       908



Casca Grossa vs Todos
Parametros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acuracia dos testes: 93.72%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        453   18  471
1         39  398  437
All      492  416  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

   

## SVM

### Tons de Cinza

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Verde vs Todos")
x_train, x_test, y_train, y_test = verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parametros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acuracia dos testes: 84.14%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        415   56  471
1         88  349  437
All      503  405  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.83      0.88      0.85       471
           1       0.86      0.80      0.83       437

    accuracy                           0.84       908
   macro avg       0.84      0.84      0.84       908
weighted avg       0.84      0.84      0.84       908



Casca Grossa vs Todos
Parametros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acuracia dos testes: 89.10%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        439   32  471
1         67  370  437
All      506  402  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.87      0.93      0.90       471
           1       0.92      0.85      

### RGB

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Verde vs Todos")
x_train, x_test, y_train, y_test = verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parametros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acuracia dos testes: 91.63%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        441   30  471
1         46  391  437
All      487  421  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.91      0.94      0.92       471
           1       0.93      0.89      0.91       437

    accuracy                           0.92       908
   macro avg       0.92      0.92      0.92       908
weighted avg       0.92      0.92      0.92       908



Casca Grossa vs Todos
Parametros: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
Acuracia dos testes: 96.70%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        463    8  471
1         22  415  437
All      485  423  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.95      0.98      0.97       471
           1       0.98      0.95      0

# **Binária: Removendo a Verde**

## K-NN

### Tons de Cinza

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRV(boa, casca_grossa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRV(boa, casca_grossa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parametros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 8, 'p': 1, 'weights': 'distance'}
Acuracia dos testes: 79.41%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        389   82  471
1        105  332  437
All      494  414  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.79      0.83      0.81       471
           1       0.80      0.76      0.78       437

    accuracy                           0.79       908
   macro avg       0.79      0.79      0.79       908
weighted avg       0.79      0.79      0.79       908



Casca Grossa vs Todos
Parametros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
Acuracia dos testes: 84.03%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        410   61  471
1         84  353  437
All      494  414  908
Relatorio de Classificacao:
              precision    recall  f1-score   supp

### RGB

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRV(boa, casca_grossa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRV(boa, casca_grossa, dano_praga, podre)
knn_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parametros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}
Acuracia dos testes: 86.78%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        420   51  471
1         69  368  437
All      489  419  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.86      0.89      0.87       471
           1       0.88      0.84      0.86       437

    accuracy                           0.87       908
   macro avg       0.87      0.87      0.87       908
weighted avg       0.87      0.87      0.87       908



Casca Grossa vs Todos
Parametros: {'leaf_size': 20, 'metric': 'chebyshev', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}
Acuracia dos testes: 93.17%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        444   27  471
1         35  402  437
All      479  429  908
Relatorio de Classificacao:
              precision    recall  f1-score   suppor

## RandomForest 

### Tons de Cinza

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRV(boa, casca_grossa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRV(boa, casca_grossa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parametros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acuracia dos testes: 82.05%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        386   85  471
1         78  359  437
All      464  444  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.83      0.82      0.83       471
           1       0.81      0.82      0.81       437

    accuracy                           0.82       908
   macro avg       0.82      0.82      0.82       908
weighted avg       0.82      0.82      0.82       908



Casca Grossa vs Todos
Parametros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acuracia dos testes: 84.58%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        409   62  471
1         78  359  437
All      487  421  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

   

### RGB

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRV(boa, casca_grossa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRV(boa, casca_grossa, dano_praga, podre)
rf_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parametros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Acuracia dos testes: 87.00%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        406   65  471
1         53  384  437
All      459  449  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.88      0.86      0.87       471
           1       0.86      0.88      0.87       437

    accuracy                           0.87       908
   macro avg       0.87      0.87      0.87       908
weighted avg       0.87      0.87      0.87       908



Casca Grossa vs Todos
Parametros: {'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Acuracia dos testes: 94.93%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        454   17  471
1         29  408  437
All      483  425  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

   

## SVM

### Tons de Cinza

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRV(boa, casca_grossa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRV(boa, casca_grossa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parametros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acuracia dos testes: 84.36%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        405   66  471
1         76  361  437
All      481  427  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.84      0.86      0.85       471
           1       0.85      0.83      0.84       437

    accuracy                           0.84       908
   macro avg       0.84      0.84      0.84       908
weighted avg       0.84      0.84      0.84       908



Casca Grossa vs Todos
Parametros: {'C': 1000, 'gamma': 1, 'kernel': 'rbf'}
Acuracia dos testes: 88.77%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        426   45  471
1         57  380  437
All      483  425  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.88      0.90      0.89       471
           1       0.89      0.87      

### RGB

In [ ]:
target_names = ['0','1']
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todosRV(boa, casca_grossa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todosRV(boa, casca_grossa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todosRV(boa, casca_grossa, dano_praga, podre)
svm_gridsearchcv(x_train, x_test, y_train, y_test, target_names)

Boa vs Todos
Parametros: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
Acuracia dos testes: 91.41%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        431   40  471
1         38  399  437
All      469  439  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.92      0.92      0.92       471
           1       0.91      0.91      0.91       437

    accuracy                           0.91       908
   macro avg       0.91      0.91      0.91       908
weighted avg       0.91      0.91      0.91       908



Casca Grossa vs Todos
Parametros: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
Acuracia dos testes: 97.36%
Matriz de Confusao:
Predito    0    1  All
Real                  
0        464    7  471
1         17  420  437
All      481  427  908
Relatorio de Classificacao:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       471
           1       0.98      0.96      0.